In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def check(filters):
    if len(filters.split()) > 1:
        n = '-'.join(filters.split())
        return n
    else:
        return filters

In [55]:
# r = requests.get("https://hh.ru/vacancies/data-scientist", headers=headers)
#
# soup = BeautifulSoup(r.text, 'lxml')
# #find_all[-1].find('span').get_text()
#
# pg = soup.find_all("a", {'class': "bloko-button",'data-qa':'pager-page'})[-1].find('span').get_text()
#
# find_all = soup.find_all("div", {'class': "vacancy-serp-item"})


def parse(filters,headers):
    columns = ["Наименование вакансии", 'Компания', "Минимальная", "Максимальная",'Валюта', 'Ссылка']
    r = requests.get(f"https://hh.ru/search/vacancy?clusters=true&area=1&ored_clusters=true&enable_snippets=true&salary=&text={check(filters)}", headers=headers)
    soup = BeautifulSoup(r.text, 'lxml')
    filters = check(filters)
    try:
        pg = soup.find_all("a", {'class': "bloko-button",'data-qa':'pager-page'})[-1].find('span').get_text()
        nums = soup.find("h1", {'class': "bloko-header-section-3",'data-qa':'bloko-header-3'}).get_text()
        df = pd.DataFrame(columns=columns,index = range(0,int(nums.split()[0])))
        k = 0
        while k != int(nums.split()[0]):
            for p in range(0,int(pg)):
                r = requests.get(f"https://hh.ru/search/vacancy?clusters=true&area=1&ored_clusters=true&enable_snippets=true&salary=&text={filters}&page={p}", headers=headers)
                soup = BeautifulSoup(r.content)
                find_all = soup.find_all("div", {'class': "vacancy-serp-item"})
                for i in range(0,len(find_all)):
                    try:
                        df.loc[k]['Наименование вакансии'] = find_all[i].find("a",{'class':"bloko-link"}).get_text()
                        df.loc[k]['Компания'] = find_all[i].find("a",{'data-qa':"vacancy-serp__vacancy-employer"}).get_text()
                        df.loc[k]['Ссылка'] = find_all[i].find("a",{'class':"bloko-link"},href = True)['href']
                        zp = find_all[i].find("div",{'class':"vacancy-serp-item__sidebar"}).get_text().split()
                        if 1 < len(zp) <= 4 :
                            df.loc[k]['Минимальная'] = None
                            df.loc[k]['Валюта']= zp[-1]
                            df.loc[k]['Максимальная'] = zp[1] + zp[2]
                        else:
                            df.loc[k]['Минимальная'] = zp[0] + zp[1]
                            df.loc[k]['Валюта']= zp[-1]
                            df.loc[k]['Максимальная'] = zp[3] + zp[4]
                    except:
                            df.loc[k]['Минимальная'] = None
                            df.loc[k]['Валюта']= None
                            df.loc[k]['Максимальная'] = None
                    k += 1
    except:
        pass
        r = requests.get(f"https://hh.ru/search/vacancy?clusters=true&area=1&ored_clusters=true&enable_snippets=true&salary=&text={check(filters)}", headers=headers)
        soup = BeautifulSoup(r.content)
        nums = soup.find("h1", {'class': "bloko-header-section-3",'data-qa':'bloko-header-3'}).get_text()
        df = pd.DataFrame(columns=columns,index = range(0,int(nums.split()[0])))
        find_all = soup.find_all("div", {'class': "vacancy-serp-item"})
        for i in range(len(find_all)):
            try:
                df.iloc[i]['Наименование вакансии'] = find_all[i].find("a",{'class':"bloko-link"}).get_text()
                df.loc[i]['Компания'] = find_all[i].find("a",{'data-qa':"vacancy-serp__vacancy-employer"}).get_text()
                df.loc[i]['Ссылка'] = find_all[i].find("a",{'class':"bloko-link"},href = True)['href']
                zp = find_all[i].find("div",{'class':"vacancy-serp-item__sidebar"}).get_text().split()
                if 1 < len(zp) <= 4 :
                    df.loc[i]['Минимальная'] = None
                    df.loc[i]['Валюта']= zp[-1]
                    df.loc[i]['Максимальная'] = zp[1] + zp[2]
                else:
                    df.loc[i]['Минимальная'] = zp[0] + zp[1]
                    df.loc[i]['Валюта']= zp[-1]
                    df.loc[i]['Максимальная'] = zp[3] + zp[4]
            except:
                df.loc[i]['Минимальная'] = None
                df.loc[i]['Валюта']= None
                df.loc[i]['Максимальная'] = None


    return df



In [56]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36", }
filters = str(input("Введите название ужной вам вакансии: "))
# filters = "data scientist"
parse(filters,headers)

,Наименование вакансии,Компания,Минимальная,Максимальная,Валюта,Ссылка
0,Senior data scientist,First data,None,150000,руб.,https://hh.ru/vacancy/49744980?from=vacancy_se...
1,Data Scientist,ООО КИБЕРБОРЕЯ,None,None,None,https://hh.ru/vacancy/49891459?from=vacancy_se...
2,Научный сотрудник / Data scientist,ООО Gero LLC,150000,230000,руб.,https://hh.ru/vacancy/50035189?from=vacancy_se...
3,Senior data scientist,Hunt Expert,None,270000,руб.,https://hh.ru/vacancy/49993197?from=vacancy_se...
4,Data Scientist (компьютерное зрение),ООО Дип Системс,200000,400000,руб.,https://hh.ru/vacancy/49557719?from=vacancy_se...
...,...,...,...,...,...,...
434,Junior/Middle NLP Data Scientist в команду Бло...,Сбер. Data Science,None,None,None,https://hh.ru/vacancy/49702248?from=vacancy_se...
435,Data Scientist,Группа компаний SRG,None,None,None,https://hh.ru/vacancy/49794918?from=vacancy_se...
436,"Lead Data Scientist, Social",Ozon Информационные технологии,None,None,None,https://hh.ru/vacancy/49748309?from=vacancy_se...
437,Data Scientist,IPONWEB Ltd (UK),None,None,None,https://hh.ru/vacancy/49782140?from=vacancy_se...
